In [23]:
from glob import glob

image_path_list = glob("images/*")
image_path_list

['images/15_31 30124013_10213858372666214_6084300428709724160_n.jpg',
 'images/5_7 46741305_10215499670297629_8916867902003478528_n.jpg',
 'images/11_0 83630985_10218853940432286_7213858926134034432_n.jpg',
 'images/NA 453632882_10229805741060457_7489649556786205200_n.jpg',
 'images/21_5 60337780_10216655216265556_1011334716533506048_n.jpg',
 'images/18_54 405741160_10228415558266756_3898063265316050459_n.jpg',
 'images/30_0 48362178_10215595765219942_1131691923644350464_n.jpg',
 'images/5_33 106138130_10220534131076002_8014245700276487117_n.jpg',
 'images/10_0 74620824_10217973801189355_6879656113399136256_n.jpg',
 'images/15_01 22712585_10213751604877086_1112620062853664594_o.jpg',
 'images/10_0 69587723_10217531521412637_6714417760765476864_n.jpg',
 'images/12_1 50458285_10215922184780227_6800444615674560512_n.jpg',
 'images/NA 122906292_10221425212992493_986834629919058392_n.jpg',
 'images/10_0 66510121_10217119054581224_47845845483126784_n.jpg',
 'images/14_8 59022840_102166047730

In [3]:
def get_distance_from_path(image_path:str):
    image_file_name = image_path.split("/")[1]
    distance = image_file_name.split(" ")[0].replace("_",".")
    if distance == "NA":
        return -1
    else:
        return float(distance)



In [4]:
import re
def get_number_before_km(text):
    pattern = r"(\d+\.?\d*)\s*km"

    match = re.search(pattern, text)
    if match:
        number_before_km = match.group(1)
        return number_before_km
    return ""

text = "The distance is a 12.5 km."
print(get_number_before_km(text))
text = "The distance is a12.5 km."
print(get_number_before_km(text))
print(get_number_before_km("15.31 km"))

12.5
12.5
15.31


In [5]:
get_number_before_km(" 15.31 km")

'15.31'

# Pytesseract

In [29]:
from PIL import Image
import pytesseract

# Load the image from the file
def get_distance_pytesseract(image_path:str)->float:

    # Use pytesseract to do OCR on the image
    extracted_text = pytesseract.image_to_string(image_path)

    distance_list= []
    # You can then filter out the distance using basic string operations
    # Assuming the distance is in the format 'Distance: 5.0 km'
    for line in extracted_text.splitlines():
        text = line.strip().lower()
        text = text.replace("/km","")
        text = text.replace("km/","")
        # if contain only km, the distance might be in previous text.
        if "km" in text:
            distance = get_number_before_km(text)
            try:
                distance = float(distance)
                distance_list.append(distance)
            except:
                print(text)
                print("not a float")
    if len(distance_list)==1:
        return distance_list[0]
    return -1

In [28]:
incorrect_count = 0
for image_path in image_path_list:
    distance = get_distance_pytesseract(image_path)
    target = get_distance_from_path(image_path)

    if distance != target:
        incorrect_count+=1
        print(image_path)
        print(target)
        print(distance)
        print("============")

print(f"score = {len(image_path_list)-incorrect_count}/{len(image_path_list)}")



images/15_31 30124013_10213858372666214_6084300428709724160_n.jpg
15.31
-1
images/5_7 46741305_10215499670297629_8916867902003478528_n.jpg
5.7
-1
images/11_0 83630985_10218853940432286_7213858926134034432_n.jpg
11.0
-1
images/21_5 60337780_10216655216265556_1011334716533506048_n.jpg
21.5
-1
images/18_54 405741160_10228415558266756_3898063265316050459_n.jpg
18.54
-1
images/30_0 48362178_10215595765219942_1131691923644350464_n.jpg
30.0
-1
images/5_33 106138130_10220534131076002_8014245700276487117_n.jpg
5.33
-1
images/10_0 74620824_10217973801189355_6879656113399136256_n.jpg
10.0
-1
images/15_01 22712585_10213751604877086_1112620062853664594_o.jpg
15.01
-1
images/10_0 69587723_10217531521412637_6714417760765476864_n.jpg
10.0
-1
images/12_1 50458285_10215922184780227_6800444615674560512_n.jpg
12.1
-1
images/NA 122906292_10221425212992493_986834629919058392_n.jpg
-1
12.16
images/10_0 66510121_10217119054581224_47845845483126784_n.jpg
10.0
-1
images/14_8 59022840_10216604773044507_879389665

# Cloud Vision API

In [ ]:
import base64
import json
import os


import functions_framework

from google.cloud import vision


vision_client = vision.ImageAnnotatorClient()


path = "images/garmin.jpg"
import io
with io.open(path, 'rb') as image_file:
        content = image_file.read()

image = vision.Image(content=content)
text_detection_response = vision_client.text_detection(image=image)

# EasyOCR

In [33]:
# Read text from an image
import easyocr
# Create an OCR reader object
reader = easyocr.Reader(["en"])
def get_distance_easyocr(image_path:str):
    result = reader.readtext(image_path)
    distance_list = []
    prev_text = ""
    for detection in result:
        text:str = detection[1]
        text = text.strip().lower()
        text = text.replace("/km","")
        text = text.replace("km/","")
        # if contain only km, the distance might be in previous text.
        if text == "km":
            text = prev_text + text
        
        if "km" in text:
            distance = get_number_before_km(text)
            try:
                distance = float(distance)
                distance_list.append(distance)
            except:
                print(text)
                print("not a float")
        prev_text = text
    if len(distance_list)==1:
        return distance_list[0]
    return -1

/Users/naveen/Library/Caches/pypoetry/virtualenvs/running-slip-reader-RFE_Jy1P-py3.11/lib/python3.11/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimenta

In [34]:
incorrect_count = 0
for image_path in image_path_list:
    distance = get_distance_easyocr(image_path)
    target = get_distance_from_path(image_path)

    if distance != target:
        incorrect_count+=1
        print(image_path)
        print(target)
        print(distance)
        print("============")

print(f"score = {len(image_path_list)-incorrect_count}/{len(image_path_list)}")



images/11_0 83630985_10218853940432286_7213858926134034432_n.jpg
11.0
0.0
images/5_33 106138130_10220534131076002_8014245700276487117_n.jpg
5.33
-1
images/15_01 22712585_10213751604877086_1112620062853664594_o.jpg
15.01
1.0
images/12_1 50458285_10215922184780227_6800444615674560512_n.jpg
12.1
4.0
images/15_18 449480091_10229550313474927_827735937888080141_n.jpg
15.18
15.48
score = 23/28
